<a href="https://colab.research.google.com/github/vamsikrishna00466/time_series/blob/main/Time%20series%20to%20ML(26-12-2021).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
#importing pandas and warnings
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 150)

#mean squred error
import numpy as np
from sklearn.metrics import mean_squared_error

from sklearn.metrics import mean_absolute_percentage_error

from sklearn.model_selection import train_test_split


In [2]:
#Initial Preprocessed data
df = pd.read_csv('/content/drive/MyDrive/SCA Projects/Forecasting /Processed_data1.csv')

In [ ]:
df

# Sku wise Forecasting -- VAR(Vector Auto Regression)

In [9]:
from statsmodels.tsa.vector_ar.var_model import VAR

In [4]:
df1 = df.groupby(by=['Month','Part No.'])['demand'].sum().reset_index()

##converting categories into columns
df2 = df1.pivot(index='Month', columns='Part No.', values='demand')

In [ ]:
#creating the train and validation set
train = df2[:int(0.8*(len(df2)))]
valid = df2[int(0.8*(len(df2))):]

#fit the model

model = VAR(endog=train)
model_fit = model.fit()

# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

In [23]:
#predictions into data frame
df_pred = pd.DataFrame(prediction, columns = df2.columns.to_list())


In [ ]:
df_pred

In [43]:
error_mape = pd.DataFrame({})
for i in valid.columns.to_list():
  er = round(mean_absolute_percentage_error(valid[i], df_pred[i]),3)
  rmse = round(np.sqrt(mean_squared_error(valid[i], df_pred[i], squared=False)),3)
  error_mape=error_mape.append({'Part No.':i,
                                'mape':er,
                                'rmse':rmse},ignore_index=True)
error_mape['mape_int'] = round(error_mape['mape']).astype(int)

In [ ]:
error_mape.sort_values(by='mape_int',ascending=False)

# categorize wise Sales forecasting --VAR

In [3]:
from statsmodels.tsa.vector_ar.var_model import VAR

In [4]:
df['sales'] = df['demand']*df['Unit Price']

df1 = df.groupby(by=['Month','Category'])['sales'].sum().reset_index()

In [ ]:
df1.tail()

In [5]:
##converting categories into columns
df2 = df1.pivot(index='Month', columns='Category', values='sales')
#resetting default index
df2 = df2.rename_axis(None).rename_axis(None, axis=1)

In [ ]:
df2.head()

In [6]:
#creating the train and validation set
train = df2[:int(0.8*(len(df2)))]
valid = df2[int(0.8*(len(df2))):]



In [ ]:
#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=train)
model_fit = model.fit()

In [ ]:
# make prediction on validation
model_fit.summary()

In [11]:
# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

In [ ]:
df_pred = pd.DataFrame(prediction, columns = df2.columns.to_list())

from sklearn.metrics import mean_absolute_percentage_error
error_mape = pd.DataFrame({})
for i in valid.columns.to_list():
  er = round(mean_absolute_percentage_error(valid[i], df_pred[i]),3)##mape calculation
  rmse = round(np.sqrt(mean_squared_error(valid[i], df_pred[i], squared=False)),3)##rmse calculation
  error_mape=error_mape.append({'Part No.':i,
                                'mape':er,
                                'rmse':rmse},ignore_index=True)

error_mape

In [ ]:
valid.plot()
df_pred.plot()

In [ ]:
## forecasting future
months_to_forecast=20
forcast_future = model_fit.forecast(model_fit.y, steps=months_to_forecast)
forcast_future1 = pd.DataFrame(prediction, columns = df2.columns.to_list())
forcast_future1.plot()

# Time series to Machine learning

In [ ]:
def data_preparing_Time_series_to_supervise_ML(df):
  #creating sales column
  df['sales'] = df['demand']*df['Unit Price']
  df1 = df.groupby(by=['Month','Category'])['sales'].sum().reset_index()
  
  ##converting categories into columns
  df2 = df1.pivot(index='Month', columns='Category', values='sales')
  #resetting default index
  df2 = df2.rename_axis(None).rename_axis(None, axis=1)
  col = df2.columns.to_list()
  df3 = df2.copy()
  df3 = round(df3).astype(int)
  #t-1lag
  
  df3['Kit(t-1)'] = df3['Kit'].shift(1)
  df3['Kit-Child Part(t-1)'] = df3['Kit-Child Part'].shift(1)
  df3['Obsolete(t-1)'] = df3['Obsolete'].shift(1)
  df3['Regular(t-1)'] = df3['Regular'].shift(1)
  #t-2 lagdf3['Kit(t-2)'] = df3['Kit'].shift(2)
  df3['Kit-Child Part(t-2)'] = df3['Kit-Child Part'].shift(2)
  df3['Obsolete(t-2)'] = df3['Obsolete'].shift(2)
  df3['Regular(t-2)'] = df3['Regular'].shift(2)
  #t-3 lag
  df3['Kit(t-3)'] = df3['Kit'].shift(3)
  df3['Kit-Child Part(t-3)'] = df3['Kit-Child Part'].shift(3)
  df3['Obsolete(t-3)'] = df3['Obsolete'].shift(3)
  df3['Regular(t-3)'] = df3['Regular'].shift(3)
  
  df3.dropna(inplace=True)
  
  df3 = df3.reset_index()
  df3['index']=pd.to_datetime(df3['index'])
  df3['month'] = df3['index'].dt.month
  df3['year'] = df3['index'].dt.year
  
  del df3['index']
  return df3, col

In [ ]:
df_ML, col = data_preparing_Time_series_to_supervise_ML(df)

# categorie Wise Sales forecasting -- With Linear Regression

In [17]:
from sklearn.linear_model import LinearRegression

In [38]:
def model_perfamance(df,i='Kit'):
  X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != i], df[i], test_size=0.20, random_state=42)
  model = LinearRegression()
  model.fit(X_train, y_train)
  y_train_predict = model.predict(X_train)
  y_test_predict = model.predict(X_test)
  print(f'MAPE Error of {i}:=--',mean_absolute_percentage_error(y_test,y_test_predict))
  return y_test_predict,y_test

In [ ]:
error_mape = pd.DataFrame({})
print('********************Forecasting error with Linear regression********************')
for i in col:
  y_test_predict,y_test = model_perfamance(df_ML,i=i)
  er = mean_absolute_percentage_error(y_test,y_test_predict)##mape calculation
  rmse = round(np.sqrt(mean_squared_error(y_test,y_test_predict, squared=False)),3)##rmse calculation
  error_mape=error_mape.append({'Part No.':i,
                                'mape':er,
                                'rmse':rmse},ignore_index=True)

round(error_mape,3)

# categorie Wise Sales forecasting -- With Decission Tree

In [49]:
from sklearn.tree import DecisionTreeRegressor


In [56]:
def model_perfamance(df,i='Kit'):
  X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != i], df[i], test_size=0.20, random_state=42)
  model = DecisionTreeRegressor(max_depth=2)
  model.fit(X_train, y_train)
  y_train_predict = model.predict(X_train)
  y_test_predict = model.predict(X_test)
  print(f'MAPE Error of {i}:=--',mean_absolute_percentage_error(y_test,y_test_predict))
  return y_test_predict,y_test

In [ ]:
error_mape = pd.DataFrame({})
print('********************Forecasting error with RandomForestRegressor********************')
for i in col:
  y_test_predict,y_test = model_perfamance(df_ML,i=i)
  er = mean_absolute_percentage_error(y_test,y_test_predict)##mape calculation
  rmse = round(np.sqrt(mean_squared_error(y_test,y_test_predict, squared=False)),3)##rmse calculation
  error_mape=error_mape.append({'Part No.':i,
                                'mape':er,
                                'rmse':rmse},ignore_index=True)

round(error_mape,3)

# categorie Wise Sales forecasting -- With Random Forest

In [33]:
from sklearn.ensemble import RandomForestRegressor


In [54]:
def model_perfamance(df,i='Kit'):
  X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != i], df[i], test_size=0.20, random_state=42)
  model = RandomForestRegressor(max_depth=2, random_state=0)
  model.fit(X_train, y_train)
  y_train_predict = model.predict(X_train)
  y_test_predict = model.predict(X_test)
  print(f'MAPE Error of {i}:=--',mean_absolute_percentage_error(y_test,y_test_predict))
  return y_test_predict,y_test

In [ ]:
error_mape = pd.DataFrame({})
print('********************Forecasting error with RandomForestRegressor********************')
for i in col:
  y_test_predict,y_test = model_perfamance(df_ML,i=i)
  er = mean_absolute_percentage_error(y_test,y_test_predict)##mape calculation
  rmse = round(np.sqrt(mean_squared_error(y_test,y_test_predict, squared=False)),3)##rmse calculation
  error_mape=error_mape.append({'Part No.':i,
                                'mape':er,
                                'rmse':rmse},ignore_index=True)

round(error_mape,3)

# categorie Wise Sales forecasting -- With XG Boost

In [46]:
from xgboost import XGBRegressor


In [47]:
def model_perfamance(df,i='Kit'):
  X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != i], df[i], test_size=0.20, random_state=42)
  model = XGBRegressor(n_estimators=10, max_depth=20, verbosity=2)#--Update required ML algorithm
  model.fit(X_train, y_train)
  y_train_predict = model.predict(X_train)
  y_test_predict = model.predict(X_test)
  print(f'MAPE Error of {i}:=--',mean_absolute_percentage_error(y_test,y_test_predict))
  return y_test_predict,y_test

In [ ]:
error_mape = pd.DataFrame({})
print('********************Forecasting error with XGBRegressor********************')
for i in col:
  y_test_predict,y_test = model_perfamance(df_ML,i=i)
  er = mean_absolute_percentage_error(y_test,y_test_predict)##mape calculation
  rmse = round(np.sqrt(mean_squared_error(y_test,y_test_predict, squared=False)),3)##rmse calculation
  error_mape=error_mape.append({'Part No.':i,
                                'mape':er,
                                'rmse':rmse},ignore_index=True)

round(error_mape,3)

# categorie Wise Sales forecasting -- With Ada Boost

In [58]:
from sklearn.ensemble import AdaBoostRegressor


In [59]:
def model_perfamance(df,i='Kit'):
  X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != i], df[i], test_size=0.20, random_state=42)
  model = AdaBoostRegressor(random_state=0, n_estimators=100)#--Update required ML algorithm
  model.fit(X_train, y_train)
  y_train_predict = model.predict(X_train)
  y_test_predict = model.predict(X_test)
  print(f'MAPE Error of {i}:=--',mean_absolute_percentage_error(y_test,y_test_predict))
  return y_test_predict,y_test

In [ ]:
error_mape = pd.DataFrame({})
print('********************Forecasting error with AdaBoostRegressor********************')
for i in col:
  y_test_predict,y_test = model_perfamance(df_ML,i=i)
  er = mean_absolute_percentage_error(y_test,y_test_predict)##mape calculation
  rmse = round(np.sqrt(mean_squared_error(y_test,y_test_predict, squared=False)),3)##rmse calculation
  error_mape=error_mape.append({'Part No.':i,
                                'mape':er,
                                'rmse':rmse},ignore_index=True)

round(error_mape,3)

# bin

In [ ]:
#importing pandas and warnings
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 150)

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

#Initial Preprocessed data
df = pd.read_csv('/content/drive/MyDrive/SCA Projects/Forecasting /Processed_data1.csv')

In [144]:
#creating sales column
df['sales'] = df['demand']*df['Unit Price']
df1 = df.groupby(by=['Month','Category'])['sales'].sum().reset_index()

##converting categories into columns
df2 = df1.pivot(index='Month', columns='Category', values='sales')
#resetting default index
df2 = df2.rename_axis(None).rename_axis(None, axis=1)

In [145]:
df3 = df2.copy()
df3 = round(df3).astype(int)

In [146]:
#t-1lag
df3['Kit(t-1)'] = df3['Kit'].shift(-1)
df3['Kit-Child Part(t-1)'] = df3['Kit-Child Part'].shift(-1)
df3['Obsolete(t-1)'] = df3['Obsolete'].shift(-1)
df3['Regular(t-1)'] = df3['Regular'].shift(-1)
#t-2 lag
df3['Kit(t-2)'] = df3['Kit'].shift(-2)
df3['Kit-Child Part(t-2)'] = df3['Kit-Child Part'].shift(-2)
df3['Obsolete(t-2)'] = df3['Obsolete'].shift(-2)
df3['Regular(t-2)'] = df3['Regular'].shift(-2)
#t-3 lag
df3['Kit(t-3)'] = df3['Kit'].shift(-3)
df3['Kit-Child Part(t-3)'] = df3['Kit-Child Part'].shift(-3)
df3['Obsolete(t-3)'] = df3['Obsolete'].shift(-3)
df3['Regular(t-3)'] = df3['Regular'].shift(-3)

In [156]:
#t-1lag
df3['Kit(t-1)'] = df3['Kit'].shift(1)
df3['Kit-Child Part(t-1)'] = df3['Kit-Child Part'].shift(1)
df3['Obsolete(t-1)'] = df3['Obsolete'].shift(1)
df3['Regular(t-1)'] = df3['Regular'].shift(1)
#t-2 lag
df3['Kit(t-2)'] = df3['Kit'].shift(2)
df3['Kit-Child Part(t-2)'] = df3['Kit-Child Part'].shift(2)
df3['Obsolete(t-2)'] = df3['Obsolete'].shift(2)
df3['Regular(t-2)'] = df3['Regular'].shift(2)
#t-3 lag
df3['Kit(t-3)'] = df3['Kit'].shift(3)
df3['Kit-Child Part(t-3)'] = df3['Kit-Child Part'].shift(3)
df3['Obsolete(t-3)'] = df3['Obsolete'].shift(3)
df3['Regular(t-3)'] = df3['Regular'].shift(3)

In [157]:
df3.dropna(inplace=True)

In [158]:
df3 = df3.reset_index()
df3['index']=pd.to_datetime(df3['index'])
df3['month'] = df3['index'].dt.month
df3['year'] = df3['index'].dt.year

In [159]:
del df3['index']

In [ ]:
for i in df2.columns.to_list():
  print(f'*********Other variables Correlation With --{i}*********')
  print(round(df3.corrwith(df3[i]).sort_values(ascending=False),3),'\n---------------\n')


In [160]:
def model_perfamance(df,i='Kit'):
  X_train, X_test, y_train, y_test = train_test_split(df3.loc[:, df.columns != i], df[i], test_size=0.20, random_state=42)
  model = XGBRegressor(n_estimators=10, max_depth=20, verbosity=2)
  model.fit(X_train, y_train)
  y_train_predict = model.predict(X_train)
  y_test_predict = model.predict(X_test)
  print(f'MAPE Error of {i}:=--',mean_absolute_percentage_error(y_test,y_test_predict))
  return y_test_predict

In [ ]:
df3.head()

In [ ]:
error_mape = pd.DataFrame({})
for i in valid.columns.to_list():
  y_test_predict = model_perfamance(df3,i=i)
  er = mean_absolute_percentage_error(y_test,y_test_predict)##mape calculation
  rmse = round(np.sqrt(mean_squared_error(y_test,y_test_predict, squared=False)),3)##rmse calculation
  error_mape=error_mape.append({'Part No.':i,
                                'mape':er,
                                'rmse':rmse},ignore_index=True)

In [ ]:
error_mape